In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00


In [3]:
!pip install sentence-transformers

In [4]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.7 MB/s eta 0:00:00


In [5]:
!pip install spacy

In [6]:
import json
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import random
import spacy
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
from google.colab import drive
import json
import pandas as pd

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to your JSON file on Google Drive
json_path = '/content/drive/My Drive/p/int.json'

with open(json_path, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."
...,...,...,...
75,fact-28,[What do I do if I'm worried about my mental h...,[The most important thing is to talk to someon...
76,fact-29,[How do I know if I'm unwell?],"[If your beliefs , thoughts , feelings or beha..."
77,fact-30,[How can I maintain social connections? What i...,"[A lot of people are alone right now, but we d..."
78,fact-31,[What's the difference between anxiety and str...,[Stress and anxiety are often used interchange...


In [9]:
# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
# Initialize stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{'again', 'our', 'very', 'in', "you'd", 'its', 'your', 'is', 'more', "shan't", 'were', 'couldn', 'doing', 'yourselves', 'any', 'd', 'most', 'mustn', 'will', 'at', "didn't", 'for', 'isn', "mightn't", 'theirs', 'do', 'm', "couldn't", 'we', 'here', 'than', 'between', 'her', 'hers', 'during', 'through', 'all', 'such', "won't", 'mightn', 'when', 'them', 'it', 'where', 'who', 'an', 'themselves', 'only', 'down', 'the', 'too', 'being', 'ma', 'what', 'be', 'been', 'him', 'or', 'no', 'once', 'they', 'didn', 'should', 'other', "doesn't", "mustn't", 'from', 'out', 'was', 'same', 'to', 'over', 'don', 'weren', 'my', 'that', 'off', 'few', 'shan', 'y', 'wasn', 'into', 'so', 'needn', 'just', 'both', 'nor', 'as', 'while', 'ours', 'he', 'have', 'aren', 'up', "you're", 'by', 're', 'a', 'why', 'then', "needn't", 'o', "shouldn't", 'she', 'his', 'own', "she's", 'ourselves', 'how', 'hadn', "hadn't", "it's", 's', 'yours', 'those', 'hasn', 't', 'against', 'with', 'there', 'll', 'shouldn', 'because', 'each', 'wh

In [12]:
# Text normalization functions
def normalize_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()     # Remove extra spaces
    return text

def expand_contractions(text):
    import contractions
    return contractions.fix(text)

def remove_stopwords(text):
    words = word_tokenize(text)
    return " ".join([word for word in words if word not in stop_words])

In [13]:
# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Initialize Google Translator for back-translation
translator = GoogleTranslator(source='en', target='fr')

# def get_spacy_synonyms(word, top_n=3):
#     """
#     Fetch synonyms for a word using spaCy's word vectors.
#     """
#     token = nlp.vocab[word]
#     if not token.has_vector:
#         return []  # Return empty list if the word has no vector representation

#     queries = [
#         (w, cosine_similarity(np.array(token.vector).reshape(1, -1),
#                               np.array(w.vector).reshape(1, -1))[0][0])
#         for w in nlp.vocab
#         if w.has_vector and w.is_lower == token.is_lower and w.prob >= -15
#     ]

#     # Sort by similarity score and take top_n results
#     synonyms = sorted(queries, key=lambda item: -item[1])
#     synonyms = [w[0].text for w in synonyms if w[0].text != word]
#     return synonyms[:top_n]

# def replace_synonyms_spacy(sentence, n=2):
#     words = sentence.split()
#     num_replaced = 0
#     for i, word in enumerate(words):
#         synonyms = get_spacy_synonyms(word)
#         if synonyms and num_replaced < n:
#             words[i] = random.choice(synonyms)
#             num_replaced += 1
#     return ' '.join(words)
from transformers import pipeline

# Initialize the fill-mask pipeline
fill_mask = pipeline("fill-mask", model="bert-base-uncased")

def get_filtered_synonyms(word, context_sentence, top_n=3):
    """
    Fetch synonyms for a word using BERT's fill-mask pipeline with improved context and filtering based on vector similarity.
    """
    mask_token = fill_mask.tokenizer.mask_token  # Ensure using the correct mask token
    masked_sentence = context_sentence.replace("[MASK]", mask_token)  # Use the tokenizer's mask token
    predictions = fill_mask(masked_sentence)

    # Load word vector for the original word
    original_token = nlp(word)
    synonyms = []

    for pred in predictions:
        candidate_word = pred['token_str']
        # Calculate similarity if the candidate is a valid word
        if candidate_word.isalpha() and candidate_word != word:
            candidate_token = nlp(candidate_word)
            if candidate_token.has_vector:
                similarity = cosine_similarity([original_token.vector], [candidate_token.vector])
                if similarity >= 0.5:  # Adjust the threshold as needed for stricter or looser matches
                    synonyms.append(candidate_word)

    return synonyms[:top_n]

def replace_synonyms_bert(sentence, n=2):
    words = sentence.split()
    num_replaced = 0
    for i, word in enumerate(words):
        # Use a more relevant context sentence for predictions
        synonyms = get_filtered_synonyms(word, context_sentence=f"The concept of {word} involves [MASK].")
        if synonyms and num_replaced < n:
            words[i] = random.choice(synonyms)
            num_replaced += 1
    return ' '.join(words)


# Back-Translation function using Google Translator
def back_translate(text, src_lang='en', mid_lang='fr'):
    """
    Translate the text from the source language to a middle language and back to the source language.
    """
    try:
        if not text or text is None:
            raise ValueError("Input text is None or empty.")

        # Forward translation
        translated = translator.translate(text, target=mid_lang)
        if not translated:
            raise ValueError("Translation returned empty result.")

        # Back-translation
        back_translated = translator.translate(translated, source=mid_lang, target=src_lang)
        if not back_translated:
            raise ValueError("Back-translation returned empty result.")

        return back_translated
    except Exception as e:
        print(f"Back-translation failed: {e}")
        return text  # Return original text if there's a failure

# Random Insertion function
def random_insertion(sentence, n=1):
    """
    Randomly insert words from the sentence into random positions.
    """
    words = sentence.split()
    for _ in range(n):
        new_word = random.choice(words)
        insert_position = random.randint(0, len(words))
        words.insert(insert_position, new_word)
    return ' '.join(words)

# Random Swap function
def random_swap(sentence, n=1):
    """
    Randomly swap the positions of words in the sentence.
    """
    words = sentence.split()
    if len(words) < 2:
        return sentence  # No swapping possible with fewer than 2 words
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# Data Augmentation function
def augment_data(patterns, num_augments=2):
    """
    Augment the input patterns with various techniques like synonym replacement,
    back-translation, random insertion, and random swap.
    """
    augmented_patterns = []
    for pattern in patterns:
        if not pattern or pattern is None:
            continue  # Skip invalid or empty patterns

        # Add original pattern
        augmented_patterns.append(pattern)

        for _ in range(num_augments):
            augmented_patterns.append(replace_synonyms_bert(pattern))
            augmented_patterns.append(back_translate(pattern))
            augmented_patterns.append(random_insertion(pattern))
            augmented_patterns.append(random_swap(pattern))

    return augmented_patterns

# Example usage
patterns = ["Hello, how are you?",
            "I love programming!",
            "Data augmentation is fun."]
augmented_patterns = augment_data(patterns, num_augments=2)

# Display augmented patterns
for pattern in augmented_patterns:
    print(pattern)
print('-----')
sentence = "I love programming!"
augmented_sentence = replace_synonyms_bert(sentence)
print(augmented_sentence)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Hello, how are you?
Hello, how are me
Bonjour, comment allez-vous ?
Hello, how how are you?
how Hello, are you?
Hello, how are me
Bonjour, comment allez-vous ?
Hello, how are you? you?
Hello, how you? are
I love programming!
i love programming
J'adore la programmation !
love I love programming!
programming! love I
i love programming
J'adore la programmation !
I love programming! programming!
love I programming!
Data augmentation is fun.
information evolution is fun.
L’augmentation des données est amusante.
Data augmentation Data is fun.
fun. augmentation is Data
information evolution is fun.
L’augmentation des données est amusante.
Data augmentation is Data fun.
Data fun. is augmentation
-----
i love programming


In [14]:
from deep_translator import GoogleTranslator

def google_translate(text, target_language='fr'):
    translated = GoogleTranslator(source='en', target=target_language).translate(text)
    return translated

# Example usage
translated_text = google_translate("Hello, how are you?", 'fr')
print(translated_text)

Bonjour comment allez-vous?


In [15]:
# Apply augmentation to each intent's patterns
df['patterns'] = df['patterns'].apply(lambda patterns: augment_data(patterns))

In [17]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [18]:
# Apply text preprocessing on patterns
df['patterns'] = df['patterns'].apply(lambda x: [normalize_text(p) for p in x])
df['patterns'] = df['patterns'].apply(lambda x: [expand_contractions(p) for p in x])
df['patterns'] = df['patterns'].apply(lambda x: [remove_stopwords(p) for p in x])

In [19]:
print(df['patterns'])

0     [hi, hi, salut, hi hi, hi, hi, salut, hi hi, h...
1     [good morning, good breakfast, bonjour, good m...
2     [good afternoon, good noon, bon aprs midi, goo...
3     [good evening, good midnight, bonne soire, goo...
4     [good night, good time, bonne nuit, good night...
                            ...                        
75    [worried mental health, worried mental health,...
76    [know unwell, know unwell, comment savoir si j...
77    [maintain social connections feel lonely, main...
78    [difference anxiety stress, equality emotion s...
79    [difference sadness depression, equality emoti...
Name: patterns, Length: 80, dtype: object


In [20]:
# Flatten patterns for consistent embeddings
expanded_data = []
for i, row in df.iterrows():
    for pattern in row['patterns']:
        expanded_data.append({
            'tag': row['tag'],
            'pattern': pattern,
            'responses': row['responses']
        })

data_expanded = pd.DataFrame(expanded_data)
print(data_expanded)

           tag                                       pattern  \
0     greeting                                            hi   
1     greeting                                            hi   
2     greeting                                         salut   
3     greeting                                         hi hi   
4     greeting                                            hi   
...        ...                                           ...   
2074   fact-32                 sadness difference depression   
2075   fact-32           contradiction depression depression   
2076   fact-32  diffrence entre la tristesse et la dpression   
2077   fact-32         difference sadness sadness depression   
2078   fact-32                 difference sadness depression   

                                              responses  
0     [Hello there. Tell me how are you feeling toda...  
1     [Hello there. Tell me how are you feeling toda...  
2     [Hello there. Tell me how are you feeling toda...  

In [21]:
# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each pattern
data_expanded['pattern_embedding'] = data_expanded['pattern'].apply(lambda x: model.encode(x))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
from sentence_transformers import SentenceTransformer
import pickle
import pandas as pd

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Assuming `data_expanded` is your DataFrame and it contains a column 'pattern'
data_expanded['pattern_embedding'] = data_expanded['pattern'].apply(lambda x: model.encode(x))

# Save the SentenceTransformer model to a local directory
model.save('sentence_transformer_model')

# Save the embeddings to a pickle file
with open('pattern_embeddings.pkl', 'wb') as f:
    pickle.dump(data_expanded['pattern_embedding'].tolist(), f)

# Optionally, save the DataFrame to a CSV file
data_expanded.to_csv('data_expanded.csv', index=False)

In [23]:
# Function to get the chatbot's response based on similarity
def get_chatbot_response(user_input):
    # Encode the user input
    user_input_embedding = model.encode(user_input)

    # Calculate cosine similarities with each pattern
    similarities = data_expanded['pattern_embedding'].apply(
        lambda x: cosine_similarity([user_input_embedding], [x])[0][0]
    )
    best_match_index = similarities.idxmax()
    response = data_expanded['responses'][best_match_index][0]
    return response

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Start a conversational loop
print("Bot: Hello! I'm here to chat. Type 'exit' anytime to end the conversation.")
while True:
    user_input = input("You: ")
    user_input = preprocess(user_input)
    # Check if the user wants to exit
    if user_input.lower() == 'exit':
        print("Bot: Goodbye! Take care.")
        break

    # Get and print the bot's response
    response = get_chatbot_response(user_input)
    print(f"Bot: {response}")

Bot: Hello! I'm here to chat. Type 'exit' anytime to end the conversation.
You: I need help
Bot: Sure. Tell me how can i assist you
You: I'm worried
Bot: It's only natural to feel this way. I'm here for you.
You: I am anxious
Bot: Don't be hard on yourself. What's the reason behind this?
You: dont know
Bot: That's no problem. I can see why you'd be stressed out about that. I can suggest you some tips to alleviate this issue. Would you like to learn more about that?
You: sure
Bot: Let's discuss further why you're feeling this way.
You: life
Bot: Duh I live in your computer
You: career
Bot: Oh I see. Tell me more
You: career, family, 
Bot: Oh I see. Tell me more
You: thats it...uncertainity
Bot: I heard you & noted it all. See you later.
You: exit
Bot: Goodbye! Take care.
